In [1]:
import pyodbc as db
import pandas as pd

In [2]:
SERVER = 'pw2-g2-y2-sqlserver.database.windows.net'
DATABASE = 'BC_Data'
USERNAME = 'group2_admin_pw2'
PASSWORD = 'AlfaBravo2'
connectionString = f'DRIVER={{SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

In [3]:
conn = db.connect(connectionString)

# Machine Learning

## Regressors

Our group has developed a regressor that predicts the ASR given certain values, here is a sample of the table used for said prediction:
|cluster_sdi|country|year|risk|ASR|SDI|
|---|---|---|---|---|---|
|Low|Bolivia|2019|Metabolic risks|1.2053777|0.566|
|Low-middle|American Samoa|2007|Dietary risks|0.4175736|0.667|
|Low|India|2018|Metabolic risks|0.668715|0.558|
|Middle|Georgia|2005|Tobacco|0.7310083|0.646|
|Low|Guatemala|2018|Low physical activit|0.0223396|0.522|
|High|Denmark|2016|Tobacco|1.2097754|0.884|
|Low|Belize|2002|Low physical activit|0.118005|0.515|
|Low|Guinea-Bissau|2000|Alcohol use|0.495362|0.249|
|Low-middle|Moldova|2018|Alcohol use|1.0012355|0.693|
|Low|Bolivia|1995|Tobacco|0.2720063|0.441|


In [5]:
# MODELS
from sklearn.ensemble import VotingRegressor #voting model https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html

from sklearn.ensemble import GradientBoostingRegressor #XGboost substitute https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor #LightGBM substitute https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html

# SCALERS
from sklearn.preprocessing import MaxAbsScaler#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html#sklearn.preprocessing.MaxAbsScaler

from sklearn.preprocessing import StandardScaler # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler

df_train =
df_test =
